To run this file & observe the outputs of our model and the results of the bert & rogue evlauation metrics simply "Run All" and make sure you are connected to a GPU (you may have to create a shortcut from the shared folder to "My Drive") prior to these steps

In [ ]:
# Install necessary huggingface packages
%%capture
!pip install transformers datasets
!pip install --upgrade accelerate
!pip install evaluate rouge_score bert_score
!pip install sentencepiece

In [ ]:
# Import packages
from transformers import pipeline, Trainer, TrainingArguments, DataCollatorForLanguageModeling, AutoTokenizer, AutoModelForSeq2SeqLM, TFAutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, T5ForConditionalGeneration
from evaluate import load
from datasets import Dataset
from transformers import DataCollatorForSeq2Seq
import numpy as np
import pandas as pd
import os
import torch
import copy
from transformers import T5ForConditionalGeneration, T5Tokenizer
import sentencepiece
from transformers import AutoTokenizer

from collections import defaultdict

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device_id = 0 if str(device) == 'cuda' else -1

## Data Pre-Processing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# folder = pd.read_csv("/content/drive/Shared With Me/folder_0/folder_1/file.csv")

Mounted at /content/drive


In [ ]:
# Load data
path = "/content/drive/My Drive/CIS 5300 Final Project/5300 Project Data/exp_data/data_email_long/"
# train_source = pd.read_csv(path + 'train_source.txt', sep = '\n')
source = []
s = open(path + 'train_source.txt', 'r')
for line in s:
  source.append(line.strip())

target = []
t = open(path + 'train_target.txt', 'r')
for line in t:
  target.append(line.strip())

train = pd.DataFrame({"Source" : source, "Target" : target})
train

,Source,Target
0,"Subject: ep strategy|||Andy: I'm fried, frazzl...",Andy expresses that he is stressed out about a...
1,"Subject: tibco layoffs, wireless cuts|||Steve:...",Steve says Tibco is planning layoffs and will ...
2,Subject: are the knicks playing tonight?|||Dou...,Doug plays a joke on Stephen and Fernand askin...
3,"Subject: func test failures|||Ravi: Sharon, I ...",Ravi tells Sharon there are some functional te...
4,"Subject: feedback from market|||Peter: Ron, I ...",The AvocadoIT team discuss problems with mobil...
...,...,...
1795,Subject: agenda for program mgmt meeting tbh 1...,Hideki is trying to update the team on the key...
1796,"Subject: cage storage|||Mark: Debbie, I need y...",Mark asks Debbie if she could help him tomorro...
1797,Subject: hp & exodus emobility initiative|||Ro...,"Ron writes to Mike, Brett, and David about the..."
1798,Subject: devices that we support today|||Rober...,Robert mentions to Dan the devices that V2.5do...


In [ ]:
# Load data
source = []
s = open(path + 'test_source.txt', 'r')
for line in s:
  source.append(line.strip())

target = []
t = open(path + 'test_target.txt', 'r')
for line in t:
  target.append(line.strip())

test = pd.DataFrame({"Source" : source, "Target" : target})
test

,Source,Target
0,"Subject: 1:30pm conference call|||Ty: Dan, Can...",Ty asks Dan if he is able to take a conference...
1,Subject: 12/20/00 need flip chart for sausalit...,Jackie alerts Yolanda and Betty she needs the ...
2,Subject: environment for respond|||Sujan: Guys...,Sujan reminds everyone they need a respond env...
3,"Subject: ert user guide|||Ruth: Piyush, I know...",Ruth requests Piyush check the status of a gui...
4,Subject: 4.5 patch for japan|||Divakar: Rajeev...,Diva tells Rajeev Japan needs a patch to fix s...
...,...,...
495,Subject: usps demo....tuesday demo!|||Ray: I ...,Ray corrects previously communicated date for ...
496,Subject: training for accenture developers in ...,Carlos will be travelling to Australia at the ...
497,Subject: ui postmortem minutes|||Mark: Here is...,Mark sends a post-mortem report to the team fo...
498,Subject: system admin training in ep japan|||R...,Richard informs people that Doug will be teach...


In [ ]:
# Load data
source = []
s = open(path + 'dev_source.txt', 'r')
for line in s:
  source.append(line.strip())

target = []
t = open(path + 'dev_target.txt', 'r')
for line in t:
  target.append(line.strip())

dev = pd.DataFrame({"Source" : source, "Target" : target})
dev

,Source,Target
0,Subject: bug 2664|||Sharon: Hi Ravi - this is ...,Sharon sends Ravi a link that isn't working. S...
1,Subject: thanks for the hospitality and courte...,Dave expresses thanks for the hospitality he r...
2,Subject: testservlet update for sp4|||Howard: ...,Howard asks about a plan for changes to TestSe...
3,Subject: andy at javaone on monday|||Andy: I'l...,Andy is at JavaOne and asks Jamie to review th...
4,Subject: avocadoit/forrester follow-up|||Ryan:...,Ryan thought event was a conference not a work...
...,...,...
244,Subject: avocadoit booth staff needed at wirel...,Debbie sends out a request for volunteers that...
245,Subject: marketing manager job description|||M...,"Marcia writes to Giselle, Barry, and Pam about..."
246,Subject: alaska bug|||Kavitha: This bug has be...,Kavitha notes that a bug has been fixed and th...
247,Subject: offline admin guide 4.0 - review|||De...,AvocadoIT completes an offline administration ...


In [ ]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the

In [ ]:
prefix = 'summarize: '
def add_summarize_prompt(df):
  inputs = [prefix + doc for doc in df["Source"]]
  df['Source'] = inputs
  model_inputs = tokenizer(inputs, max_length = 512, truncation = True)
  labels = tokenizer(text_target=df["Target"], max_length = 32, truncation = True)
  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

ds_train = Dataset.from_pandas(train)
tokenized_train = ds_train.map(add_summarize_prompt, batched = True)

ds_dev = Dataset.from_pandas(dev)
tokenized_dev = ds_dev.map(add_summarize_prompt, batched = True)

ds_test = Dataset.from_pandas(test)
tokenized_test = ds_test.map(add_summarize_prompt, batched = True)

Map:   0%|          | 0/1800 [00:00<?, ? examples/s]

Map:   0%|          | 0/249 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

## Strong Baseline: Fine-Tuned T5

In [ ]:
def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

  prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
  result["gen_len"] = np.mean(prediction_lens)

  return {k: round(v, 4) for k, v in result.items()}

In [ ]:
# ft_model = AutoModelForSeq2SeqLM.from_pretrained('t5-base')
ft_model = T5ForConditionalGeneration.from_pretrained('t5-base').to('cuda')

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
checkpoint = "t5-base"
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)
rouge = load("rouge")
bertscore = load("bertscore")

In [ ]:
# Train model
training_args = Seq2SeqTrainingArguments(
    output_dir="ft_t5_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=ft_model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_dev,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,2.375630,0.343900,0.114300,0.289400,0.289600,18.931700
2,No log,2.233910,0.358200,0.123900,0.300900,0.301100,18.911600
3,No log,2.193108,0.362500,0.129800,0.307800,0.307700,18.971900
4,No log,2.181629,0.363800,0.129700,0.307700,0.307400,18.931700


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=452, training_loss=2.398046983026825, metrics={'train_runtime': 538.5113, 'train_samples_per_second': 13.37, 'train_steps_per_second': 0.839, 'total_flos': 4325998794670080.0, 'train_loss': 2.398046983026825, 'epoch': 4.0})

In [ ]:
trainer.save_model('./ft_t5_model')
ft_summarizer = pipeline("summarization", model="/content/ft_t5_model")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Fine tune conditional generation model to use GPUs

In [ ]:
# Assuming 'test' is your DataFrame and 'Source' is the column with text to summarize

# Load the model and tokenizer
model = T5ForConditionalGeneration.from_pretrained('./ft_t5_model').to('cuda')

def batch_summarize(texts, batch_size=8):
    summaries = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]

        # Tokenize the texts and prepare the input tensor
        inputs = tokenizer(batch_texts, padding=True, truncation=True, max_length=512, return_tensors='pt').to('cuda')

        # Generate summaries
        with torch.no_grad():
            summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=50, early_stopping=True)

        # Decode and add to the list
        batch_summaries = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
        summaries.extend(batch_summaries)

    return summaries

# Apply batch summarization
test['Predicted'] = batch_summarize(tokenized_test['Source'])

## Model Evaluation & Example Summaries

In [ ]:
# Token indices sequence length is longer than the specified maximum sequence length for this model (645 > 512). Running this sequence through the model will result in indexing errors
ft_rouge_results = rouge.compute(predictions = test['Predicted'], references = test['Target'])
ft_berts_results = bertscore.compute(predictions = test['Predicted'], references = test['Target'], lang = 'en')

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
ft_rouge_results

{'rouge1': 0.2870647508228293,
 'rouge2': 0.09132341307731788,
 'rougeL': 0.2137893218412717,
 'rougeLsum': 0.21375501967454538}

In [ ]:
ft_berts_results

{'precision': [0.9407638907432556,
  0.930474042892456,
  0.8703195452690125,
  0.9114016890525818,
  0.8493371605873108,
  0.8602691888809204,
  0.8559213280677795,
  0.841033935546875,
  0.8205703496932983,
  0.8847477436065674,
  0.837889552116394,
  0.907230794429779,
  0.8921365737915039,
  0.9107031226158142,
  0.8881582021713257,
  0.9067090749740601,
  0.8897205591201782,
  0.9231147170066833,
  0.8526104688644409,
  0.8452301025390625,
  0.8783143758773804,
  0.8778411149978638,
  0.8461068868637085,
  0.8851515650749207,
  0.9295998811721802,
  0.8655718564987183,
  0.889028012752533,
  0.9068009853363037,
  0.8730008602142334,
  0.86479651927948,
  0.869309663772583,
  0.9090678095817566,
  0.8732782602310181,
  0.8570042252540588,
  0.9369297027587891,
  0.8383347988128662,
  0.9195458889007568,
  0.8631126284599304,
  0.8587318062782288,
  0.8748814463615417,
  0.9027845859527588,
  0.886388897895813,
  0.8766815662384033,
  0.8963247537612915,
  0.9009075164794922,
  0.91

In [ ]:
avg_bert_precision = np.mean(ft_berts_results["precision"])
avg_bert_accuracy = np.mean(ft_berts_results["recall"])
avg_bert_f1 = np.mean(ft_berts_results["f1"])
print("Precision:", avg_bert_precision)
print("Recall:", avg_bert_accuracy)
print("F1-Score", avg_bert_f1)

Precision: 0.8856813507080078
Recall: 0.8580387581586838
F1-Score 0.8715098963975907


In [ ]:
# Example 1
print(test['Predicted'][0])
print(tokenized_test['Source'][0])
print(test['Target'][0])

Ty asks Dan if he can sit in on a conference call with Accenture today at 1:30pm to talk about Symbol devices, barcoding into an application, and a few other questions regarding an opportunity at Visa.
summarize: Subject: 1:30pm conference call|||Ty: Dan, Can you sit in on a call with Accenture today at 1:30pm to talk about Symbol devices, barcoding into an application, and a few other questions regarding an opportunity at Visa? Thanks, Ty|||Dan: Yes but need more tech info on our symbol peripheral capabilities so may need Amitabh.|||Ty: I think you may be able to cover this initial discussion without Amitabh. We'ss dial you in or come over to Amit's office. Ty|||Dan: I'm working from home today, so you can call my mobile of give me a dial-in ph#. Dan						USERNAME@DOMAIN.COM Director of Sales Engineering PHONENUMBER m HTTP://LINK w|||Dan: are you going to call my mobile phone or do you have a dial-in ph# ? Dan						USERNAME@DOMAIN.COM Director of Sales Engineering PHONENUMBER m HTTP:/

In [ ]:
# Example 2
print(test['Predicted'][1])
print(tokenized_test['Source'][1])
print(test['Target'][1])

Jackie asks Yolanda to clean out the fridge. Yolanda asks if she will need a flip chart today.
summarize: Subject: 12/20/00 need flip chart for sausalito conf rm.|||Jackie: Also, Please clean out the fridge. Thank you. Jackie|||Yolanda: are you going to need flip chart today? and yes, i will clean the refrigerator this morning, remember, i asked you to let me know when the meeting was over the other day, so i could clean the refrigerator.|||Jackie: Thanks Yoli!! Sorry, I couldn't remember if I had sent this one or not. I appreciate your efforts and hard work to help me out!!! :)|||Yolanda: you have sent me this message 3 times already and iam currently looking for one|||Jackie: Sorry!!! Thanks again.
Jackie alerts Yolanda and Betty she needs the flip chart for the Sausalito conf rm and requests the fridge to be cleaned. Yolanda asks Jackie if she will need the flip chart today, and assures her that she will clean the fridge this morning. Jackie apologizes for sending these messages thr

In [ ]:
# Example 3
print(test['Predicted'][2])
print(tokenized_test['Source'][2])
print(test['Target'][2])

Sujan tells the group that they need a "standard destop" hardware configuration which they can use as the server.
summarize: Subject: environment for respond|||Sujan: Guys, We need a Respond environment. For that I need to order a "standard destop" hardware configuration which we can use as the server. As it will only be used as a server, we don't need the fancy monitor (ie 15" will be ok). Please order one for my group. Let me know the availability and if you need more specific details. Thanks, Sujan|||Sujan: Guys, We need a Respond environment. For that I need to order a "standard destop" hardware configuration which we can use as the server. As it will only be used as a server, we don't need the fancy monitor (ie 15" will be ok). Please order one for my group. Let me know the availability and if you need more specific details. Thanks, Sujan|||Marek: Sujan, I'm not sure if you already took care of that, but the external IP is very important (if we want to test Portal from a real phon

In [ ]:
test.to_csv('/content/drive/My Drive/t5_ft_data_long.csv')